## Get Download URLs from SARVIEWS

In order to get a Download URL for a specific granule in SARVIEWS, you will need two things:
1. usgs_event_id
2. granule_name

In [59]:
##Example usgs_event_id and granule_name

usgs_event_id = 'ak20076877'
granule_name  = 'S1AA_20180504T161046_20180528T161048_VVP024_INT80_G_weF_5E24'

In [ ]:
usgs_event_id = input('USGS Event ID:')
granule_name  = input('Granulate Name:')

First, you will need to make a GET request to https://gm3385dq6j.execute-api.us-west-2.amazonaws.com/events in order to retrieve a list of all of the events known to SARVIEWS. The response will be a list of the form:
```json
[
    {
        "depth": "",
        "description": "",
        "event_date": "",
        "event_id": "99",
        "event_type": "quake",
        "magnitude": ,
        "processing_timeframe": {
            "end": "",
            "start": ""
        },
        "usgs_event_id": "ak20076877",
        "wkt": ""
    },
    ...
]
```

In [ ]:
import json
import requests

event_list_res = requests.get('https://gm3385dq6j.execute-api.us-west-2.amazonaws.com/events')

event_list = event_list_res.json()
event_obj  = next((item for item in event_list if item["usgs_event_id"] == usgs_event_id), None)
event_id   = event_obj['event_id']

event_get_res = requests.get(f'https://gm3385dq6j.execute-api.us-west-2.amazonaws.com/events/{event_id}')

event_get_list = event_get_res.json()
event_obj      = next((item for item in event_get_list["products"] if item["files"]["product_name"][:-4] == granule_name), None)

download_link = event_obj['files']['product_url']

200

In [ ]:
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

with urlopen(
        download_link
    ) as response:
        with ZipFile(BytesIO(response.read())) as file:
            file.extractall()